In [1]:
import random
n=10
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=0.5

In [2]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return alpha*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [3]:
import itertools
import matplotlib.pyplot as plt
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
#     for i in range(n):
#         plt.scatter(ps[i][0],ps[i][1],c='k')
#         connect_x=[ps[res[i]][0],ps[res[(i+1)%len(ps)]][0]]
#         connect_y=[ps[res[i]][1],ps[res[(i+1)%len(ps)]][1]]
#         plt.plot(connect_x,connect_y, c='red')
    return res

In [4]:
import itertools
import matplotlib.pyplot as plt
def heuristic1(tsp):
    label={}
    operation={}
    flyop={}
    driveop={}
    tsptotal=0
    for i in tsp:
        label[i]='simple'
        tsptotal=tsptotal+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    n=len(label)
    tspdtotal=tsptotal
    ms=[1]
    while max(ms)>0:
        ms=[]
        for i in range(n):
            if label[tsp[i]]=='combined' or label[tsp[i]]=='drone' or label[tsp[i]]=='truck':
                ms.append(-float('inf'))
            else:
                mfsi=t_cost(V[tsp[i-1]],V[tsp[i]])+t_cost(V[tsp[i]],V[tsp[(i+1)%n]])-max(d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]]),t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]]))
                ms.append(mfsi)
        for i in range(n):
            if label[tsp[i]]=='combined':
                if label[tsp[i-1]]==('combined'or'simple'):
                    j=i+1
                    while label[tsp[j%n]]!='combined':
                        if label[tsp[j%n]]=='drone':
                            dronenode=j%n
                        if label[tsp[(j+1)%n]]=='combined':
                            pls=t_cost(V[tsp[i-1]],V[tsp[i]])+operation[(tsp[dronenode],tsp[i],tsp[(j+1)%n])]-max(d_cost(V[tsp[i-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]]),driveop[(tsp[dronenode],tsp[i],tsp[(j+1)%n])]+t_cost(V[tsp[i-1]],V[tsp[i]]))
                            ms.append(pls)
                            break
                        j=j+1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(n):
            if label[tsp[i]]=='combined':
                if label[tsp[(i+1)%n]]==('combined'or'simple'):
                    j=i-1
                    while label[tsp[(j+n)%n]]!='combined':
                        if label[tsp[(j+n)%n]]=='drone':
                            dronenode=(j+n)%n
                        if label[tsp[j-1]]=='combined':
                            pls=t_cost(V[tsp[i]],V[tsp[(i+1)%n]])+operation[(tsp[dronenode],tsp[j-1],tsp[i])]-max(d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(i+1)%n]]),driveop[(tsp[dronenode],tsp[j-1],tsp[i])]+t_cost(V[tsp[i]],V[tsp[(i+1)%n]]))
                            ms.append(pls)
                            break
                        j=j-1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(len(ms)):
            if ms[i]==max(ms) and ms[i]>0:
                tspdtotal=tspdtotal-ms[i]
                if i//n==0:
                    label[tsp[i]]='drone'
                    label[tsp[i-1]]='combined'
                    label[tsp[(i+1)%n]]='combined'
                    flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]])
                    driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]])
                    operation[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=max(flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])],driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])])
                elif i//n==1:
                    label[tsp[i%n]]='truck'
                    label[tsp[(i-1)%n]]='combined'
                    j=(i+1)%n
                    while label[tsp[j%n]]!='combined':
                        if label[tsp[j%n]]=='drone':
                            dronenode=j
                        if label[tsp[(j+1)%n]]=='combined':
                            flyop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])]=d_cost(V[tsp[(i-1)%n]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]])
                            driveop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])]=driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]+t_cost(V[tsp[(i-1)%n]],V[tsp[i%n]])
                            operation[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])]=max(flyop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])],driveop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])])
                            break
                        j=j+1
                elif i//n==2:
                    label[tsp[i%n]]='truck'
                    label[tsp[(i+1)%n]]='combined'
                    j=(i-1)%n
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[j-1]]=='combined':
                            flyop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])]=d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(i+1)%n]])
                            driveop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])]=driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])]+t_cost(V[tsp[i%n]],V[tsp[(i+1)%n]])
                            operation[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])]=max(flyop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])],driveop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])])
                            break
                        j=j-1
#     for i in range(n):
#         plt.scatter(V[tsp[i]][0],V[tsp[i]][1],c='k')
#         if label[tsp[i]]=='combined':
#             j=(i+1)%n
#             truck=[]
#             while label[tsp[j]]!='combined':
#                 if label[tsp[j]]=='drone':
#                     dronenode=j
#                 if label[tsp[j]]=='truck':
#                     truck.append(j)
#                 if label[tsp[(j+1)%n]]=='combined':
#                     plt.plot([V[tsp[i]][0],V[tsp[dronenode]][0]],[V[tsp[i]][1],V[tsp[dronenode]][1]],c='blue')
#                     plt.plot([V[tsp[(j+1)%n]][0],V[tsp[dronenode]][0]],[V[tsp[(j+1)%n]][1],V[tsp[dronenode]][1]],c='blue')
#                     if len(truck)==0:
#                         plt.plot([V[tsp[(j+1)%n]][0],V[tsp[i]][0]],[V[tsp[(j+1)%n]][1],V[tsp[i]][1]],c='red')
#                     else:
#                         plt.plot([V[tsp[i]][0],V[tsp[truck[0]]][0]],[V[tsp[i]][1],V[tsp[truck[0]]][1]],c='red')
#                         plt.plot([V[tsp[(j+1)%n]][0],V[tsp[truck[-1]]][0]],[V[tsp[(j+1)%n]][1],V[tsp[truck[-1]]][1]],c='red')
#                         for k in range(0,len(truck)-1):
#                             plt.plot([V[truck[k]][0],V[truck[k+1]][0]],[V[truck[k]][1],V[truck[k+1]][1]],c='red')
#                         truck=[]
#                     break
#                 j=(j+1)%n
#             if label[tsp[j]]=='combined':
#                 plt.plot([V[tsp[j]][0],V[tsp[i]][0]],[V[tsp[j]][1],V[tsp[i]][1]],c='red')
                    
            
            
    return tspdtotal
            


In [5]:
import itertools
import matplotlib.pyplot as plt
def heuristic1original(tsp):
    label={}
    operation={}
    flyop={}
    driveop={}
    tsptotal=0
    for i in tsp:
        label[i]='simple'
        tsptotal=tsptotal+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    n=len(label)
    tspdtotal=tsptotal
    ms=[1]
    while 'simple' in label and max(ms)>0:
        ms=[]
        for i in range(n):
            if label[tsp[i]]=='combined' or label[tsp[i]]=='drone' or label[tsp[i]]=='truck':
                ms.append(-float('inf'))
            else:
                mfsi=t_cost(V[tsp[i-1]],V[tsp[i]])+t_cost(V[tsp[i]],V[tsp[(i+1)%n]])-max(d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]]),t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]]))
                ms.append(mfsi)
        for i in range(n):
            if label[tsp[i]]=='combined':
                if label[tsp[i-1]]==('combined'or'simple'):
                    j=i+1
                    while label[tsp[j%n]]!='combined':
                        if label[tsp[j%n]]=='drone':
                            dronenode=j%n
                        if label[tsp[(j+1)%n]]=='combined':
                            pls=t_cost(V[tsp[i-1]],V[tsp[i]])+operation[(tsp[dronenode],tsp[i],tsp[(j+1)%n])]-max(d_cost(V[tsp[i-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]]),driveop[(tsp[dronenode],tsp[i],tsp[(j+1)%n])]+t_cost(V[tsp[i-1]],V[tsp[i]]))
                            ms.append(pls)
                            break
                        j=j+1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(n):
            if label[tsp[i]]=='combined':
                if label[tsp[(i+1)%n]]==('combined'or'simple'):
                    j=i-1
                    while label[tsp[(j+n)%n]]!='combined':
                        if label[tsp[(j+n)%n]]=='drone':
                            dronenode=(j+n)%n
                        if label[tsp[j-1]]=='combined':
                            pls=t_cost(V[tsp[i]],V[tsp[(i+1)%n]])+operation[(tsp[dronenode],tsp[j-1],tsp[i])]-max(d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(i+1)%n]]),driveop[(tsp[dronenode],tsp[j-1],tsp[i])]+t_cost(V[tsp[i]],V[tsp[(i+1)%n]]))
                            ms.append(pls)
                            break
                        j=j-1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(len(ms)):
            if ms[i]==max(ms) and ms[i]>0:
                tspdtotal=tspdtotal-ms[i]
                if i//n==0:
                    label[tsp[i]]='drone'
                    label[tsp[i-1]]='combined'
                    label[tsp[(i+1)%n]]='combined'
                    flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]])
                    driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]])
                    operation[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=max(flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])],driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])])
                elif i//n==1:
                    label[tsp[i%n]]='truck'
                    label[tsp[(i-1)%n]]='combined'
                    j=(i+1)%n
                    while label[tsp[j%n]]!='combined':
                        if label[tsp[j%n]]=='drone':
                            dronenode=j
                        if label[tsp[(j+1)%n]]=='combined':
                            flyop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])]=d_cost(V[tsp[(i-1)%n]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]])
                            driveop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])]=driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]+t_cost(V[tsp[(i-1)%n]],V[tsp[i%n]])
                            operation[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])]=max(flyop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])],driveop[(tsp[dronenode],tsp[(i-1)%n],tsp[(j+1)%n])])
                            break
                        j=j+1
                elif i//n==2:
                    label[tsp[i%n]]='truck'
                    label[tsp[(i+1)%n]]='combined'
                    j=(i-1)%n
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[j-1]]=='combined':
                            flyop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])]=d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(i+1)%n]])
                            driveop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])]=driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])]+t_cost(V[tsp[i%n]],V[tsp[(i+1)%n]])
                            operation[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])]=max(flyop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])],driveop[(tsp[dronenode],tsp[j-1],tsp[(i+1)%n])])
                            break
                        j=j-1
#     for i in range(n):
#         plt.scatter(V[tsp[i]][0],V[tsp[i]][1],c='k')
#         if label[tsp[i]]=='combined':
#             j=(i+1)%n
#             truck=[]
#             while label[tsp[j]]!='combined':
#                 if label[tsp[j]]=='drone':
#                     dronenode=j
#                 if label[tsp[j]]=='truck':
#                     truck.append(j)
#                 if label[tsp[(j+1)%n]]=='combined':
#                     plt.plot([V[tsp[i]][0],V[tsp[dronenode]][0]],[V[tsp[i]][1],V[tsp[dronenode]][1]],c='blue')
#                     plt.plot([V[tsp[(j+1)%n]][0],V[tsp[dronenode]][0]],[V[tsp[(j+1)%n]][1],V[tsp[dronenode]][1]],c='blue')
#                     if len(truck)==0:
#                         plt.plot([V[tsp[(j+1)%n]][0],V[tsp[i]][0]],[V[tsp[(j+1)%n]][1],V[tsp[i]][1]],c='red')
#                     else:
#                         plt.plot([V[tsp[i]][0],V[tsp[truck[0]]][0]],[V[tsp[i]][1],V[tsp[truck[0]]][1]],c='red')
#                         plt.plot([V[tsp[(j+1)%n]][0],V[tsp[truck[-1]]][0]],[V[tsp[(j+1)%n]][1],V[tsp[truck[-1]]][1]],c='red')
#                         for k in range(0,len(truck)-1):
#                             plt.plot([V[truck[k]][0],V[truck[k+1]][0]],[V[truck[k]][1],V[truck[k+1]][1]],c='red')
#                         truck=[]
#                     break
#                 j=(j+1)%n
#             if label[tsp[j]]=='combined':
#                 plt.plot([V[tsp[j]][0],V[tsp[i]][0]],[V[tsp[j]][1],V[tsp[i]][1]],c='red')
                    
            
            
    return tspdtotal
            


In [ ]:
import random
n=10
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=0.5

In [10]:
count=0
n=10
res=[]
while count <=1000:
    V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
    lis =two_opt(V)
    if heuristic1(lis) < heuristic1original(lis):
        res.append(1)
    count =count+1
print(len(res))

1001


In [11]:
heuristic1original(two_opt(V))

197.24316324715602

In [12]:
heuristic1(two_opt(V))

145.01215124326887